<a href="https://colab.research.google.com/github/BhekiMabheka/Explore/blob/main/How_Probability_Calibration_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How Probability Calibration Works

Probability calibration is the process of calibrating an ML model to return the true likelihood of an event. This is necessary when we need the probability of the event in question rather than its classification.

In [ ]:
# Create dataset of classification task with many redundant and few informative features

import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, precision_score, recall_score, f1_score
np.random.seed(0)

X, y = datasets.make_classification(n_samples=100000, n_features=20, n_informative=2, n_redundant=2)

train_samples = 100
X_train = X[:train_samples]
X_test = X[train_samples:]
y_train = y[:train_samples]
y_test = y[train_samples:]

model_rfc = RandomForestClassifier()
model_rfc.fit(X_train, y_train)
y_pred_rfc = model_rfc.predict(X_test)

print('Random Forest: ')
print("Precision: %0.2f" % precision_score(y_test, y_pred_rfc))
print("Recall: %0.2f" % recall_score(y_test, y_pred_rfc))
print("F1: %0.2f\n" % f1_score(y_test, y_pred_rfc))

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

print('Logistic Regression: ')
print("Precision: %0.2f" % precision_score(y_test, y_pred_lr))
print("Recall: %0.2f" % recall_score(y_test, y_pred_lr))
print("F1: %0.2f" % f1_score(y_test, y_pred_lr))

The random forest classifier (RFC) got an F1 score of 0.89, which is not bad. The logistic regression performed just a bit worse than RF with a score of 0.85. But how well calibrated are they?

---